In [1]:
import os
import cv2
import numpy as np
import pandas as pd
from keras.optimizers import SGD, Adam
from keras.layers import *
from keras.models import Sequential, Model
from sklearn.model_selection import *
from sklearn.metrics import *
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.applications import MobileNet

C:\Users\hp\AppData\Local\Temp\ipykernel_8904\2504913834.py:4: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_images():
  path = "Dataset\PCOSGen-train\PCOSGen-train\images"
  images = os.listdir(path)
  x = []
  for img in images:
    temp = cv2.imread(os.path.join(path ,img))
    twmp=cv2.resize(temp,(256,256))

    x.append(temp)

  return x

In [3]:
df = pd.read_excel("Dataset\class_label.xlsx")
df.head()

,imagePath,Healthy
0,100image13.jpg,1
1,100image2.jpg,1
2,100image65.jpg,0
3,100image71.jpg,0
4,100image83.jpg,0


In [4]:
image_names = df['imagePath'].values
df_filtered = df[df['imagePath'].isin(image_names)]


In [5]:
x=load_images()

In [6]:
images = []
labels = []
for index, row in df_filtered.iterrows():
    image_name = row['imagePath']
    label = row['Healthy']
    image_index = np.where(np.array(image_names) == image_name)[0][0]
    image = x[image_index]
    resized_image = cv2.resize(image, (256, 256))
    images.append(resized_image)
    labels.append(label)

In [7]:
images = np.array(images)
labels = np.array(labels)

In [8]:
x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)


In [9]:
y_train_reshaped = np.expand_dims(np.expand_dims(y_train, axis=-1), axis=-1)
y_test_reshaped = np.expand_dims(np.expand_dims(y_test, axis=-1), axis=-1)

batch_size=16
epochs=10


In [10]:
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

class_labels=np.unique(y_train)
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=class_labels, y=y_train)
class_weights=dict(zip(np.unique(y_train), class_weights)),class_weights

In [11]:
from sklearn.utils import class_weight
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

class_labels=np.unique(y_train)
class_weights = class_weight.compute_class_weight(class_weight='balanced',classes=class_labels, y=y_train)
class_weights=dict(zip(np.unique(y_train), class_weights)),class_weights

In [15]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Input, BatchNormalization, Activation, Concatenate, AveragePooling2D
from keras.applications.densenet import DenseNet121
def dense():
    base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

    for layer in base_model.layers:
        layer.trainable = False

    x = base_model.output
    x = Flatten()(x)
    x = Dense(1024, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.5)(x)
    # x = Dense(512, activation='relu')(x)
    # x = BatchNormalization()(x)
    # x = Dropout(0.5)(x)




    predictions = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=base_model.input, outputs=predictions)

    return model

In [16]:
densenet_model=dense()
densenet_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [17]:
densenet_model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=11, batch_size=16)

Epoch 1/11


  6/160 [>.............................] - ETA: 1:21:04 - loss: 1.7313 - accuracy: 0.5208

KeyboardInterrupt: 

In [18]:
from keras.applications import DenseNet121, ResNet50
from keras.layers import Dense, Dropout, Flatten, Input, concatenate
from keras.models import Model, Sequential

# Load pre-trained models
base_model1 = DenseNet121(include_top=False, weights='imagenet', input_shape=(256, 256, 3))
base_model2 = ResNet50(include_top=False, weights='imagenet', input_shape=(256, 256, 3))

# Make sure the layers of the pre-trained models are not trainable
for layer in base_model1.layers:
    layer.trainable = False

for layer in base_model2.layers:
    layer.trainable = False

# Define a simple ANN model
def create_ann():
    model = Sequential()
    model.add(Dense(512, activation='relu', input_shape=(256, 256, 3)))
    model.add(Dropout(0.5))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    return model

# Create the ANN model
ann_model = create_ann()

# Define the ensemble model
def ensemble_model():
    inputs = Input(shape=(256, 256, 3))
    x1 = base_model1(inputs)
    x1 = Flatten()(x1)
    x2 = base_model2(inputs)
    x2 = Flatten()(x2)
    x3 = ann_model(inputs)
    x = concatenate([x1, x2, x3])
    x = Dense(1024, activation='relu')(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=inputs, outputs=predictions)
    return model

# Create the ensemble model
ensemble = ensemble_model()


94765736/94765736 [==============================] - 612s 6us/step


ResourceExhaustedError: {{function_node __wrapped__StatelessRandomUniformV2_device_/job:localhost/replica:0/task:0/device:CPU:0}} OOM when allocating tensor with shape[33554432,256] and type float on /job:localhost/replica:0/task:0/device:CPU:0 by allocator mklcpu [Op:StatelessRandomUniformV2] name: 

In [ ]:
# Compile the model
ensemble.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fit the model to your data
history = ensemble.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=10, batch_size=32)


In [ ]:
from keras.preprocessing.image import ImageDataGenerator

# Define your data directories
train_dir = 'path_to_your_train_data'
val_dir = 'path_to_your_validation_data'

# Create a data generator for the training data
train_datagen = ImageDataGenerator(rescale=1./255)  # normalize pixel values to [0,1]
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(256, 256),  # resize images to 256 x 256
    batch_size=32,
    class_mode='binary'  # for binary classification
)

# Create a data generator for the validation data
val_datagen = ImageDataGenerator(rescale=1./255)  # normalize pixel values to [0,1]
val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(256, 256),  # resize images to 256 x 256
    batch_size=32,
    class_mode='binary'  # for binary classification
)

# Now you can use these generators with the `fit` method
history = ensemble.fit(
    train_generator,
    steps_per_epoch=100,  # number of batches per epoch, depends on your data
    epochs=10,
    validation_data=val_generator,
    validation_steps=50  # number of batches per validation epoch, depends on your data
)
